In [13]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import warnings
# warnings.filterwarnings('ignore')

# Add src to path
sys.path.insert(0, 'src')

# GARCH imports
from models.garch import fit_garch, rolling_garch_forecast, get_model_params

# LSTM imports
from models.lstm import rolling_lstm_forecast


# # Evaluation
from eval.metrics import qlike
from eval.backtests import vol_target_weights, run_backtest
from eval.plots import (
    plot_volatility_comparison,
    plot_forecast_errors,
    plot_scatter_comparison,
    plot_backtest_results
)

# # Research analysis
from research.garch_analysis import VolatilityComparison, stylized_facts_summary



In [14]:
def prepare_lstm_data(df_raw):
    """
    Prepare comprehensive features for LSTM

    Parameters:
    -----------
    df_raw : pd.DataFrame
        Raw OHLCV data

    Returns:
    --------
    features_df : pd.DataFrame
        Feature DataFrame with 'rv' target
    """
    print("Creating features for LSTM...")

    # Create all features
    features = create_volatility_features(df_raw)

    print(f"✓ Created {len(features.columns)} features")
    print(f"✓ Valid samples: {len(features)}")

    return features

def load_data(ticker='SPY', start='2015-01-01', end='2024-10-28'):
    """Load data from Yahoo Finance"""
    import yfinance as yf
    print(f"\nDownloading {ticker} data from {start} to {end}...")

    df = yf.download(ticker, start=start, end=end, progress=False)
    df.columns = [c.lower() for c in df.columns]

    # Basic features
    df['ret'] = df['close'].pct_change()
    df['log_ret'] = np.log(df['close'] / df['close'].shift(1))

    # Realized volatility
    df['rv'] = realized_vol_from_daily(df)

    print(f"✓ Downloaded {len(df)} days of data\n")
    return df.dropna()

In [15]:
    #df = load_data(ticker, start_date, end_date)
    df = pd.read_parquet('data/processed/AAPL_features.parquet')
    # Split into train/test
    train_end = '2020-12-31'
    test_start = '2021-01-01'

    print(f"Training period: {df.loc[:train_end].index[0].date()} to {df.loc[:train_end].index[-1].date()}")
    print(f"Testing period:  {df.loc[test_start:].index[0].date()} to {df.loc[test_start:].index[-1].date()}")


Training period: 2015-02-06 to 2020-12-31
Testing period:  2021-01-04 to 2025-10-23


In [16]:
df

,open,high,low,close,volume,ret,logret,rv_target,rv_lag1,rv_lag5,rv_lag22,ret_1d,ret_5d,ret_22d,vol_22d,weekday,month,ticker
date,,,,,,,,,,,,,,,,,,
2015-02-06,26.738340,26.789581,26.388572,26.495508,174826400,-0.008421,-0.008457,0.410099,0.475384,0.520643,0.544024,0.007138,0.002578,0.005923,0.341151,4,2,AAPL
2015-02-09,26.410848,26.698236,26.384114,26.671503,155559200,0.006642,0.006620,0.303013,0.410099,0.503857,0.537368,-0.008421,0.003820,0.005536,0.344102,0,2,AAPL
2015-02-10,26.771752,27.212862,26.769525,27.183899,248034000,0.019211,0.019029,0.289036,0.303013,0.453461,0.524961,0.006642,0.002639,0.005200,0.342822,1,2,AAPL
2015-02-11,27.350982,27.829964,27.290831,27.821053,294247200,0.023439,0.023168,0.261613,0.289036,0.403446,0.509409,0.019211,0.006448,0.004327,0.326246,2,2,AAPL
2015-02-12,28.083948,28.400300,27.974785,28.173061,297898000,0.012653,0.012573,0.296280,0.261613,0.347829,0.494561,0.023439,0.009602,0.005344,0.332295,3,2,AAPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-10-17,248.020004,253.380005,247.270004,252.289993,49147000,0.019559,0.019371,0.423456,0.543770,0.510702,0.408093,-0.007580,-0.005115,0.001857,0.246999,4,10,AAPL
2025-10-20,255.889999,264.380005,255.630005,262.239990,90483000,0.039439,0.038681,0.482868,0.423456,0.499332,0.410919,0.019559,0.005701,0.002586,0.254155,0,10,AAPL
2025-10-21,261.880005,265.290009,261.829987,262.769989,46695900,0.002021,0.002019,0.469364,0.482868,0.499416,0.415121,0.039439,0.011640,0.004590,0.281434,1,10,AAPL


In [18]:
    print("=" * 80)
    print("1. ANALYZING STYLIZED FACTS")
    print("=" * 80)

    facts = stylized_facts_summary(df['ret'])

    print(f"\n✓ Volatility clustering: {'YES' if facts['volatility_clustering'] else 'NO'}")
    print(f"✓ Heavy tails:           {'YES' if facts['heavy_tails'] else 'NO'}")
    print(f"✓ Mean return:           {facts['mean_return']*252*100:.2f}% annualized")
    print(f"✓ Volatility:            {facts['volatility']*np.sqrt(252)*100:.2f}% annualized")
    print()

    # =================================================================
    # 3. GARCH MODELS
    # =================================================================
    print("=" * 80)
    print("2. FITTING GARCH MODELS")
    print("=" * 80)

    print("\nGenerating GARCH forecasts (this may take a few minutes)...")

    # Generate GARCH forecasts
    garch_fcst = rolling_garch_forecast(
        df['ret'],
        window=1260,  # 5 years
        kind='garch',
        refit_freq=20  # Monthly
    )

    egarch_fcst = rolling_garch_forecast(
        df['ret'],
        window=1260,
        kind='egarch',
        refit_freq=20
    )

    print(f"✓ GARCH forecasts:  {len(garch_fcst)} predictions")
    print(f"✓ EGARCH forecasts: {len(egarch_fcst)} predictions")
    print()

    # =================================================================
    # 4. LSTM MODEL
    # =================================================================
    print("=" * 80)
    print("3. TRAINING LSTM MODEL")
    print("=" * 80)

    # Prepare features
    features_df = df

    # Select features for LSTM
    feature_cols = select_lstm_features(features_df, correlation_threshold=0.95)
    print(f"\nSelected {len(feature_cols)} features for LSTM after correlation filtering")
    print(f"Features: {', '.join(feature_cols[:10])}...")
    print()

    print("Generating LSTM forecasts (this will take several minutes)...")
    print("Note: LSTM refits every 20 days with early stopping for efficiency")
    print()

    # Generate LSTM forecasts
    lstm_fcst = rolling_lstm_forecast(
        data=features_df,
        target_col='rv',
        feature_cols=feature_cols,
        seq_len=30,  # 30-day lookback
        train_window=756,  # 3 years
        refit_freq=20,  # Monthly refit
        lstm_hidden=64,
        lstm_layers=2,
        epochs=50,  # Max epochs (early stopping typically stops around 20-30)
        batch_size=32,
        verbose=True
    )

    print(f"\n✓ LSTM forecasts: {len(lstm_fcst)} predictions")
    print()

    # =================================================================
    # 5. BASELINE METHODS
    # =================================================================
    print("=" * 80)
    print("4. GENERATING BASELINE FORECASTS")
    print("=" * 80)

    # Historical volatility
    hist_vol = df['ret'].rolling(20).std() * np.sqrt(252)

    # EWMA
    ewma_vol = df['ret'].ewm(halflife=10).std() * np.sqrt(252)

    print("✓ Historical volatility (20-day)")
    print("✓ EWMA (halflife=10)")
    print()

    # =================================================================
    # 6. OUT-OF-SAMPLE EVALUATION
    # =================================================================
    print("=" * 80)
    print("5. OUT-OF-SAMPLE FORECAST EVALUATION")
    print("=" * 80)

    # Get test period data
    test_rv = df.loc[test_start:, 'rv']

    # Create comparison object
    comp = VolatilityComparison(test_rv)
    comp.add_forecast('GARCH(1,1)', garch_fcst)
    comp.add_forecast('EGARCH(1,1)', egarch_fcst)
    comp.add_forecast('LSTM', lstm_fcst)
    comp.add_forecast('Historical Vol', hist_vol.loc[test_start:])
    comp.add_forecast('EWMA', ewma_vol.loc[test_start:])

    # Compute metrics
    metrics = comp.compute_metrics()

    print("\n" + "=" * 80)
    print(metrics[['Model', 'N', 'RMSE', 'MAE', 'QLIKE', 'R²', 'Bias']].to_string(index=False))
    print("=" * 80)

    # Find best models
    best_qlike = metrics.nsmallest(1, 'QLIKE').iloc[0]
    best_r2 = metrics.nlargest(1, 'R²').iloc[0]

    print(f"\n✓ Best QLIKE: {best_qlike['Model']} ({best_qlike['QLIKE']:.4f})")
    print(f"✓ Best R²:    {best_r2['Model']} ({best_r2['R²']:.4f})")
    print()

    # Statistical comparison
    print("=" * 80)
    print("STATISTICAL COMPARISON: GARCH vs LSTM")
    print("=" * 80)

    if 'GARCH(1,1)' in comp.forecasts and 'LSTM' in comp.forecasts:
        dm_result = comp.diebold_mariano_test('GARCH(1,1)', 'LSTM', loss_func='qlike')
        print(f"\nDiebold-Mariano Test (QLIKE loss):")
        print(f"  H0: Equal predictive accuracy")
        print(f"  Test statistic: {dm_result['DM_statistic']:.3f}")
        print(f"  P-value:        {dm_result['p_value']:.4f}")
        print(f"  Result:         {'Significantly different' if dm_result['significant'] else 'Not significantly different'}")
        if dm_result['significant']:
            print(f"  Better model:   {dm_result['better_model']}")
        print()

    # =================================================================
    # 7. VISUALIZATIONS
    # =================================================================
    print("=" * 80)
    print("6. GENERATING VISUALIZATIONS")
    print("=" * 80)

    # Forecast comparison plot
    fig1 = plot_volatility_comparison(
        test_rv,
        {
            'GARCH': garch_fcst.loc[test_start:],
            'EGARCH': egarch_fcst.loc[test_start:],
            'LSTM': lstm_fcst,
            'EWMA': ewma_vol.loc[test_start:]
        },
        title=f"{ticker} Volatility Forecasts: GARCH vs LSTM",
        highlight_periods={'COVID': ('2020-02-01', '2020-05-01')} if test_start <= '2020-02-01' else None
    )
    plt.savefig('comparison_forecasts.png', dpi=300, bbox_inches='tight')
    print("✓ Saved: comparison_forecasts.png")
    plt.close()

    # Forecast errors
    fig2 = plot_forecast_errors(
        test_rv,
        {
            'GARCH': garch_fcst,
            'EGARCH': egarch_fcst,
            'LSTM': lstm_fcst
        }
    )
    plt.savefig('comparison_errors.png', dpi=300, bbox_inches='tight')
    print("✓ Saved: comparison_errors.png")
    plt.close()

    # Scatter plots
    fig3 = plot_scatter_comparison(
        test_rv,
        {
            'GARCH': garch_fcst,
            'EGARCH': egarch_fcst,
            'LSTM': lstm_fcst
        }
    )
    plt.savefig('comparison_scatter.png', dpi=300, bbox_inches='tight')
    print("✓ Saved: comparison_scatter.png")
    plt.close()

    print()

    # =================================================================
    # 8. ECONOMIC VALUE: VOLATILITY TARGETING
    # =================================================================
    print("=" * 80)
    print("7. EVALUATING ECONOMIC VALUE (VOLATILITY TARGETING)")
    print("=" * 80)

    print("\nRunning backtests...")

    test_returns = df.loc[test_start:, 'ret']

    backtests = {}

    # GARCH strategy
    if len(garch_fcst.loc[test_start:]) > 0:
        garch_weights = vol_target_weights(garch_fcst.loc[test_start:], sigma_star=0.10, w_max=2.0)
        backtests['GARCH'] = run_backtest(test_returns, garch_weights, tc_bps=5, slip_bps=1)

    # EGARCH strategy
    if len(egarch_fcst.loc[test_start:]) > 0:
        egarch_weights = vol_target_weights(egarch_fcst.loc[test_start:], sigma_star=0.10, w_max=2.0)
        backtests['EGARCH'] = run_backtest(test_returns, egarch_weights, tc_bps=5, slip_bps=1)

    # LSTM strategy
    if len(lstm_fcst) > 0:
        lstm_weights = vol_target_weights(lstm_fcst, sigma_star=0.10, w_max=2.0)
        backtests['LSTM'] = run_backtest(test_returns, lstm_weights, tc_bps=5, slip_bps=1)

    # EWMA strategy
    ewma_weights = vol_target_weights(ewma_vol.loc[test_start:], sigma_star=0.10, w_max=2.0)
    backtests['EWMA'] = run_backtest(test_returns, ewma_weights, tc_bps=5, slip_bps=1)

    # Buy & Hold
    bh_weights = pd.Series(1.0, index=test_returns.index)
    backtests['Buy & Hold'] = run_backtest(test_returns, bh_weights, tc_bps=5, slip_bps=1)

    # Results table
    bt_summary = pd.DataFrame({
        'Strategy': [name for name in backtests.keys()],
        'Sharpe Ratio': [bt['sharpe'] for bt in backtests.values()],
        'Max DD (%)': [bt['max_drawdown'] * 100 for bt in backtests.values()],
        'Final Equity': [bt['equity'].iloc[-1] for bt in backtests.values()],
        'Avg Turnover': [bt['turnover'] for bt in backtests.values()]
    })

    print("\n" + "=" * 80)
    print(bt_summary.to_string(index=False))
    print("=" * 80)

    # Find best strategy
    best_sharpe = bt_summary.nlargest(1, 'Sharpe Ratio').iloc[0]
    best_dd = bt_summary.nsmallest(1, 'Max DD (%)').iloc[0]

    print(f"\n✓ Highest Sharpe: {best_sharpe['Strategy']} ({best_sharpe['Sharpe Ratio']:.2f})")
    print(f"✓ Lowest DD:      {best_dd['Strategy']} ({best_dd['Max DD (%)']:.2f}%)")
    print()

    # Plot backtest results
    fig4 = plot_backtest_results(backtests)
    plt.savefig('comparison_backtests.png', dpi=300, bbox_inches='tight')
    print("✓ Saved: comparison_backtests.png")
    plt.close()

    # =================================================================
    # 9. FINAL SUMMARY
    # =================================================================
    print("\n" + "=" * 80)
    print("RESEARCH SUMMARY")
    print("=" * 80)

    print(f"\n📊 Data: {ticker} ({start_date} to {end_date})")
    print(f"   Training: {df.loc[:train_end].index[0].date()} to {df.loc[:train_end].index[-1].date()}")
    print(f"   Testing:  {df.loc[test_start:].index[0].date()} to {df.loc[test_start:].index[-1].date()}")

    print(f"\n✓ Stylized Facts:")
    print(f"   Volatility clustering: {'Confirmed' if facts['volatility_clustering'] else 'Not found'}")
    print(f"   Heavy tails: {'Confirmed' if facts['heavy_tails'] else 'Not found'}")

    print(f"\n✓ Forecast Accuracy (Test Period):")
    print(f"   Best QLIKE: {best_qlike['Model']} = {best_qlike['QLIKE']:.4f}")
    print(f"   Best R²:    {best_r2['Model']} = {best_r2['R²']:.4f}")

    print(f"\n✓ Economic Value (Sharpe Ratios):")
    for _, row in bt_summary.iterrows():
        print(f"   {row['Strategy']:15s}: {row['Sharpe Ratio']:>6.2f}")

    print(f"\n✓ Key Findings:")

    # Compare GARCH vs LSTM
    garch_metrics = metrics[metrics['Model'] == 'GARCH(1,1)'].iloc[0] if 'GARCH(1,1)' in metrics['Model'].values else None
    lstm_metrics = metrics[metrics['Model'] == 'LSTM'].iloc[0] if 'LSTM' in metrics['Model'].values else None

    if garch_metrics is not None and lstm_metrics is not None:
        qlike_improvement = (garch_metrics['QLIKE'] - lstm_metrics['QLIKE']) / garch_metrics['QLIKE'] * 100

        if lstm_metrics['QLIKE'] < garch_metrics['QLIKE']:
            print(f"   • LSTM outperforms GARCH by {abs(qlike_improvement):.1f}% (QLIKE)")
        else:
            print(f"   • GARCH outperforms LSTM by {abs(qlike_improvement):.1f}% (QLIKE)")

    # Compare Sharpe ratios
    if 'GARCH' in backtests and 'LSTM' in backtests:
        sharpe_diff = backtests['LSTM']['sharpe'] - backtests['GARCH']['sharpe']
        if sharpe_diff > 0.1:
            print(f"   • LSTM strategy has higher Sharpe (+{sharpe_diff:.2f})")
        elif sharpe_diff < -0.1:
            print(f"   • GARCH strategy has higher Sharpe (+{abs(sharpe_diff):.2f})")
        else:
            print(f"   • GARCH and LSTM strategies have similar Sharpe ratios")

    # Compare to buy & hold
    if 'LSTM' in backtests:
        bh_sharpe = backtests['Buy & Hold']['sharpe']
        lstm_sharpe = backtests['LSTM']['sharpe']
        improvement = lstm_sharpe - bh_sharpe
        print(f"   • Vol-targeting improves Sharpe by {improvement:.2f} vs Buy & Hold")

    print(f"\n✓ Generated Figures:")
    print(f"   • comparison_forecasts.png  - Forecast time series")
    print(f"   • comparison_errors.png     - Forecast errors")
    print(f"   • comparison_scatter.png    - Actual vs predicted")
    print(f"   • comparison_backtests.png  - Backtest results")

    print("\n" + "=" * 80)
    print("COMPARISON COMPLETE")
    print("=" * 80)
    print()

1. ANALYZING STYLIZED FACTS

✓ Volatility clustering: YES
✓ Heavy tails:           YES
✓ Mean return:           25.48% annualized
✓ Volatility:            28.98% annualized

2. FITTING GARCH MODELS

Generating GARCH forecasts (this may take a few minutes)...
✓ GARCH forecasts:  1435 predictions
✓ EGARCH forecasts: 1435 predictions

3. TRAINING LSTM MODEL


NameError: name 'select_lstm_features' is not defined